In [1]:
!pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: e:\major project\.venv\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 


#### Merge the data

In [1]:
import numpy as np
balanced_dir= "C:/Users/Sai Shashank Anchuri/OneDrive/Desktop/Major-Project/Data/"

# Load balanced data
# data_2_3 = np.load(balanced_dir+"balanced_2-3.npy")
# data_3_4 = np.load(balanced_dir+"balanced_3-4.npy")
# data_4_5 = np.load(balanced_dir+"balanced_4-5.npy")
# data_5_6 = np.load(balanced_dir+"balanced_5-6.npy")
# data_6_plus = np.load(balanced_dir+"balanced_6+.npy")

# Merge all categories
# X = np.concatenate([data_2_3, data_3_4, data_4_5, data_5_6, data_6_plus])
# y = np.concatenate([
#     np.full(len(data_2_3), 2),
#     np.full(len(data_3_4), 3),
#     np.full(len(data_4_5), 4),
#     np.full(len(data_5_6), 5),
#     np.full(len(data_6_plus), 6)
# ])

# np.save(balanced_dir+"final/X_final.npy", X)
# np.save(balanced_dir+"final/y_final.npy", y)

#### Normalize the waveforms

In [2]:
import numpy as np
balanced_dir= "C:/Users/Sai Shashank Anchuri/OneDrive/Desktop/Major-Project/Data/"
def normalize_waveform(waveform):
    return (waveform - np.min(waveform)) / (np.max(waveform) - np.min(waveform)) * 2 - 1
X = np.load(r"C:\Users\Sai Shashank Anchuri\OneDrive\Desktop\Major-Project\Data\final\X_final.npy")
X = np.array([normalize_waveform(w) for w in X])
np.save(balanced_dir+"final/X_normalized.npy", X)

#### Split the data

In [3]:
y = np.load(r"C:\Users\Sai Shashank Anchuri\OneDrive\Desktop\Major-Project\Data\final\y_final.npy")
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

np.save(balanced_dir+"split/X_train.npy", X_train)
np.save(balanced_dir+"split/X_val.npy", X_val)
np.save(balanced_dir+"split/X_test.npy", X_test)
np.save(balanced_dir+"split/y_train.npy",y_train)
np.save(balanced_dir+"split/y_val.npy", y_val)
np.save(balanced_dir+"split/y_test.npy",y_test)

#### Make and save class weights

In [4]:
# y = np.load(balanced_dir+"final/y_final.npy")
# from sklearn.utils.class_weight import compute_class_weight
import json
# weights = compute_class_weight("balanced", classes=np.unique(y), y=y)
# class_weights = dict(enumerate(weights))

# print("Class Weights:", class_weights)
# # Save class weights to a JSON file
# with open(balanced_dir+"/models/class_weights.json", "w") as json_file:
#     json.dump(class_weights, json_file, indent=4)

# print("Class weights saved to class_weights.json")

# Load class weights from JSON
with open(balanced_dir+"/models/class_weights.json", "r") as json_file:
    loaded_class_weights = json.load(json_file)

# Convert keys back to integers (JSON stores dictionary keys as strings)
loaded_class_weights = {int(k): v for k, v in loaded_class_weights.items()}

print("Loaded Class Weights:", loaded_class_weights)


Loaded Class Weights: {0: 0.56566, 1: 0.56566, 2: 0.7744877002418876, 3: 6.78792, 4: 38.74383561643835}


#### Load all the training and testing data

In [5]:
import numpy as np
X_train = np.load(balanced_dir+"split/X_train.npy")
X_val = np.load(balanced_dir+"split/X_val.npy")
X_test = np.load(balanced_dir+"split/X_test.npy")
y_train = np.load(balanced_dir+"split/y_train.npy")
y_val = np.load(balanced_dir+"split/y_val.npy")
y_test = np.load(balanced_dir+"split/y_test.npy")

#### Train Model

In [7]:
import numpy as np
from tensorflow.keras import layers, models, callbacks

# Define CNN-GRU model
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(1000, 3)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.GRU(64, return_sequences=True),
    layers.GRU(32),
    layers.Dense(1, activation='linear')  # Magnitude regression output
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define callback to save the model after every epoch
checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="models/10_epochs_model/magnitude_model_epoch_{epoch:02d}.h5",  # Save as epoch number
    save_best_only=False,  # Set to True if you want to save only the best model
    save_weights_only=False,
    verbose=1
)

# Train the model and save after each epoch
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
                    epochs=10, batch_size=32, class_weight=loaded_class_weights,
                    callbacks=[checkpoint_callback])

# Save the final trained model separately
model.save("models/10_epochs_model/magnitude_prediction_model_final.h5")

# Evaluate the final model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae}")

Epoch 1/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 8.0785 - mae: 1.0539
Epoch 1: saving model to models/10_epochs_model/magnitude_model_epoch_01.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2747s 1s/step - loss: 8.0762 - mae: 1.0539 - val_loss: 1.2242 - val_mae: 0.9464
Epoch 2/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.7624 - mae: 0.9432
Epoch 2: saving model to models/10_epochs_model/magnitude_model_epoch_02.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2775s 1s/step - loss: 2.7624 - mae: 0.9432 - val_loss: 1.2388 - val_mae: 0.9507
Epoch 3/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.5988 - mae: 0.8701
Epoch 3: saving model to models/10_epochs_model/magnitude_model_epoch_03.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2797s 2s/step - loss: 2.5988 - mae: 0.8701 - val_loss: 1.0318 - val_mae: 0.8440
Epoch 4/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.5182 - mae: 0.8116
Epoch 4: saving model to models/10_epochs_model/magnitude_model_epoch_04.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2808s 2s/step - loss: 2.5182 - mae: 0.8116 - val_loss: 1.1206 - val_mae: 0.9046
Epoch 5/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.4458 - mae: 0.7891
Epoch 5: saving model to models/10_epochs_model/magnitude_model_epoch_05.h5


...

1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2806s 2s/step - loss: 2.3619 - mae: 0.7514 - val_loss: 1.0653 - val_mae: 0.8672
Epoch 8/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.4011 - mae: 0.7745
Epoch 8: saving model to models/10_epochs_model/magnitude_model_epoch_08.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2778s 1s/step - loss: 2.4011 - mae: 0.7745 - val_loss: 0.6450 - val_mae: 0.6476
Epoch 9/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.2771 - mae: 0.7084
Epoch 9: saving model to models/10_epochs_model/magnitude_model_epoch_09.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2700s 1s/step - loss: 2.2771 - mae: 0.7084 - val_loss: 0.8151 - val_mae: 0.7708
Epoch 10/10
1857/1857 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 2.2851 - mae: 0.7198
Epoch 10: saving model to models/10_epochs_model/magnitude_model_epoch_10.h5


1857/1857 ━━━━━━━━━━━━━━━━━━━━ 2711s 1s/step - loss: 2.2852 - mae: 0.7198 - val_loss: 1.3260 - val_mae: 0.9699


398/398 ━━━━━━━━━━━━━━━━━━━━ 133s 335ms/step - loss: 1.3438 - mae: 0.9800
Test MAE: 0.9720828533172607


#### Test the model on a single waveform

In [5]:
import time
import numpy as np
# X_test = np.load(r"E:\Major Project\Data\Split\X_test.npy")
# y_test = np.load(r"E:\Major Project\Data\Split\y_test.npy")
from tensorflow.keras.models import load_model

# Load the model
model_1_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_01.h5")
model_2_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_02.h5")
model_3_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_03.h5")
model_4_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_04.h5")
model_5_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_05.h5")
model_6_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_06.h5")
model_7_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_07.h5")
model_8_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_08.h5")
model_9_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_09.h5")
model_10_epochs = load_model("E:\Major Project\Code\models\10_epochs_model\magnitude_model_epoch_10.h5")

models = [model_1_epochs, model_2_epochs, model_3_epochs, model_4_epochs, model_5_epochs,
          model_6_epochs, model_7_epochs, model_8_epochs, model_9_epochs, model_10_epochs]

# Select one sample (e.g., first sample)
sample_index = 0  # Change this to test a different sample
X_sample = X_test[sample_index].reshape(1, 6000, 3)  # Reshape for model input
y_actual = y_test[sample_index]

print(f"Actual Magnitude: {y_actual:.2f}")
predictions = []
# Predict magnitude with all the 10 models
for i in models:
    start_time = time.time()
    y_pred = i.predict(X_sample)[0][0]
    predictions.append(y_pred)
    end_time = time.time()
    print("Model-{}-epochs:".format(models.index(i)+1))
    print(f"Predicted Magnitude: {y_pred:.2f}")
    print(f"Time taken for prediction: {end_time - start_time:.4f} seconds")

# Identify the model with closest prediction
closest_prediction_index = np.argmin(np.abs(np.array(predictions) - y_actual))
closest_model = models[closest_prediction_index]



ImportError: cannot import name 'AutoGraphError' from partially initialized module 'tensorflow.python.autograph.impl.api' (most likely due to a circular import) (e:\Major Project\.venv\lib\site-packages\tensorflow\python\autograph\impl\api.py)

In [15]:
X_test.shape

(12728, 6000, 3)

In [ ]:
y_test.shape

(12728,)

In [9]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
     -------- ------------------------------- 1.3/6.2 MB 6.7 MB/s eta 0:00:01
     ------------------ --------------------- 2.9/6.2 MB 7.6 MB/s eta 0:00:01
     ------------------------------ --------- 4.7/6.2 MB 7.9 MB/s eta 0:00:01
     ---------------------------------------- 6.2/6.2 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------------------------- 536.2/536.2 kB 18.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 10.1 MB/s eta 0:04:31
   ---------------------------------------- 0.0/2.7 GB 9.1 MB/s eta 0:05:00
   ---------------------------------------- 0.0/2.7 GB 9.3 MB/s eta 0:04:54
   ---------------------------------------- 0.0/2.7 GB 9.4 MB/s eta 0:04:51
   ----------------------------

In [10]:
import torch

# Check if ROCm (Linux) or DirectML (Windows) detects GPU
device = torch.device("cuda" if torch.cuda.is_available() else 
                      "mps" if torch.backends.mps.is_available() else 
                      "cpu")
print(f"Using device: {device}")

Using device: cpu


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Detect device (AMD GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load preprocessed training data
X_train = np.load(balanced_dir+"split/X_train.npy")
y_train = np.load(balanced_dir+"split/y_train.npy")

X_val = np.load(balanced_dir+"split/X_val.npy")
y_val = np.load(balanced_dir+"split/y_val.npy")

# Convert NumPy arrays to PyTorch tensors and move to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)

# Create DataLoader for batching
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define CNN-GRU Model
class CNN_GRU_Model(nn.Module):
    def _init_(self):
        super(CNN_GRU_Model, self)._init_()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.gru = nn.GRU(input_size=128, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 1)  # Output layer for magnitude prediction

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x, _ = self.gru(x)
        x = self.fc(x[:, -1, :])  # Take last GRU output
        return x

# Instantiate model and move it to GPU
model = CNN_GRU_Model().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)  # Move batch to GPU
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch.unsqueeze(1))  # Reshape y to match output
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation loss
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.unsqueeze(1))
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

# Save model
torch.save(model.state_dict(), "magnitude_prediction_model.pth")

Using device: cpu


NameError: name 'balanced_dir' is not defined

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, callbacks

# Define CNN-GRU model (as you have already done)
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(1000, 3)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.GRU(64, return_sequences=True),
    layers.GRU(32),
    layers.Dense(1, activation='linear')  # Magnitude regression output
])

# Plot the model
plot_model(model, to_file='model_diagram.png', show_shapes=True, show_layer_names=True)


ModuleNotFoundError: No module named 'PIL'

In [3]:
!pip install pydot

     ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
     ------------------------------------ - 102.4/107.7 kB 2.9 MB/s eta 0:00:01
     -------------------------------------- 107.7/107.7 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
